In [11]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [12]:
#reading vaclogger first measurement file
vaclog1=pd.read_csv("EGA testing vaclog before config change", sep="\t")
vaclog2=pd.read_csv("EGA testing vaclog after config change", sep="\t")
vaclog = pd.concat([vaclog1,vaclog2],ignore_index=True)
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,V_heater,I_emission,I_grid,100mbar inj,1mbar inj,DUAL inj line
0,NaN,2023-04-27 14:45:40,0.0,-0.050953,1.440000e-06,1.970000e-07,7.874000e-07,1.710000e-06,-0.000687,0.0,92.592,65.544,0.000015,NaN,NaN,NaN,NaN,NaN
1,NaN,2023-04-27 14:45:47,7.0,-0.050855,1.400000e-06,1.920000e-07,7.614000e-07,1.272000e-06,-0.000724,0.0,92.368,65.656,0.000012,NaN,NaN,NaN,NaN,NaN
2,NaN,2023-04-27 14:45:54,14.0,-0.051448,1.370000e-06,1.890000e-07,7.478000e-07,1.127000e-06,-0.000685,0.0,92.368,65.320,0.000014,NaN,NaN,NaN,NaN,NaN
3,NaN,2023-04-27 14:46:01,21.0,-0.051150,1.330000e-06,1.840000e-07,7.284000e-07,1.032000e-06,-0.000687,0.0,92.480,65.207,0.000010,NaN,NaN,NaN,NaN,NaN
4,NaN,2023-04-27 14:46:09,29.0,-0.051275,1.290000e-06,1.800000e-07,7.076000e-07,9.797000e-07,-0.000687,0.0,92.592,65.320,0.000014,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7736,NaN,2023-04-28 09:06:21,66041.0,NaN,2.380000e-09,1.050000e-09,4.996000e-09,6.127000e-07,NaN,2.0,10.887,252.748,0.000023,NaN,NaN,-0.058892,1.0755,0.000002
7737,NaN,2023-04-28 09:06:30,66050.0,NaN,2.390000e-09,1.060000e-09,4.996000e-09,6.135000e-07,NaN,2.0,11.223,252.636,0.000021,NaN,NaN,-0.058275,1.0755,0.000002
7738,NaN,2023-04-28 09:06:38,66058.0,NaN,2.370000e-09,1.060000e-09,4.996000e-09,6.157000e-07,NaN,2.0,10.999,252.636,0.000028,NaN,NaN,-0.058177,1.0755,0.000002
7739,NaN,2023-04-28 09:06:47,66067.0,NaN,2.370000e-09,1.050000e-09,4.996000e-09,6.171000e-07,NaN,2.0,10.887,252.524,0.000034,NaN,NaN,-0.058252,1.0755,0.000002


In [13]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("Ega testing hvlog", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv["I_bias"]=[element * 1e-6 for element in hv["IMon3"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em,I_bias
0,2023-04-27 14:44:28,27/04/2023,0.000,14:44:28.893,499.94,100.1,1000.06,1000.12,1000.12,499.88,...,51.6467,16.5879,37.6031,25.4494,0.0015,-0.0139,NaN,1.530000e-08,2.420000e-08,0.000052
1,2023-04-27 14:44:30,27/04/2023,1.204,14:44:30.097,499.94,100.1,1000.06,1000.12,1000.14,499.88,...,51.7362,16.6679,37.6789,25.5284,0.0018,-0.0139,NaN,1.530000e-08,2.420000e-08,0.000052
2,2023-04-27 14:44:31,27/04/2023,2.408,14:44:31.301,499.94,100.1,1000.06,1000.12,1000.14,499.88,...,51.7238,16.6577,37.6703,25.5189,0.0018,-0.0139,NaN,1.530000e-08,2.410000e-08,0.000052
3,2023-04-27 14:44:32,27/04/2023,3.603,14:44:32.496,499.94,100.1,1000.06,1000.12,1000.14,499.88,...,51.7218,16.6579,37.6710,25.5195,0.0018,-0.0139,NaN,1.530000e-08,2.410000e-08,0.000052
4,2023-04-27 14:44:33,27/04/2023,4.808,14:44:33.701,499.94,100.1,1000.06,1000.12,1000.14,499.88,...,51.7313,16.6714,37.6853,25.5383,0.0018,-0.0139,NaN,1.530000e-08,2.410000e-08,0.000052


In [14]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
print(hv_comm)
hv_comm.index

[nan 'grid transparent' 'grid repelling' 'ega test finished']
                datetime            Comment
108  2023-04-27 14:46:39   grid transparent
109  2023-04-27 14:46:40   grid transparent
110  2023-04-27 14:46:41   grid transparent
111  2023-04-27 14:46:42   grid transparent
628  2023-04-27 14:57:04     grid repelling
629  2023-04-27 14:57:05     grid repelling
630  2023-04-27 14:57:06     grid repelling
631  2023-04-27 14:57:07     grid repelling
2660 2023-04-27 15:37:54  ega test finished
2661 2023-04-27 15:37:55  ega test finished
2662 2023-04-27 15:37:56  ega test finished
2663 2023-04-27 15:37:58  ega test finished


Int64Index([108, 109, 110, 111, 628, 629, 630, 631, 2660, 2661, 2662, 2663], dtype='int64')

In [15]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [16]:
#print hv comments of first file
print(pd.unique(vaclog["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

[nan 'ega moved in' 'config change' 'transfer complete'
 'inj volume prepared']
                    Time        Live comments
113  2023-04-27 14:59:29         ega moved in
840  2023-04-27 16:32:50        config change
956  2023-04-27 16:50:49    transfer complete
7717 2023-04-28 09:03:38  inj volume prepared


Int64Index([113, 840, 956, 7717], dtype='int64')

### Plotting

In [17]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [18]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Baryon 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Baryon 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='Dual exp')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution during EGA testing")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')  

#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/Baryons_EGA testing.png')
plt.show()

In [19]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.plot(hv["datetime"],hv["I_bias"],marker=".", markersize=4,label='Bias current')

plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("EGA testing HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/HiVolta_EGA testing.png')
plt.show()


In [20]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution during EGA testing")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp_EGA testing.png')
plt.show()

OverflowError: (34, 'Result too large')